In [1]:
import sys
from pathlib import Path
sys.path.append((Path.cwd().parent.parent).__str__())

In [2]:
from pyqpanda_alg.QAOA import default_circuits
import numpy as np
from pyqpanda3.core import QProg

In [4]:
prog = QProg(4)
qubits = prog.qubits()
circuit = default_circuits.parity_partition_xy_mixer(qubits, np.pi/4)
print(circuit)


          ┌──────────────┐        ┌───────────────┐        ┌───────────────┐ ┌──────────────┐       ┌────┐ ┌───────────────┐       ┌────┐ ┌───────────────┐ 
q_0:  |0>─┤RX(1.57079633)├ ───*── ┤RX(-1.57079633)├ ───*── ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ──────┤CNOT├ ┤RZ(-1.57079633)├ ──────┤CNOT├ ┤RX(-1.57079633)├ 
          ├──────────────┤ ┌──┴─┐ ├───────────────┤ ┌──┴─┐ ├───────────────┤ ├──────────────┤       └──┬─┘ ├───────────────┤       └──┬─┘ ├───────────────┤ 
q_1:  |0>─┤RX(1.57079633)├ ┤CNOT├ ┤RZ(-1.57079633)├ ┤CNOT├ ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ───*─────┼── ┤RX(-1.57079633)├ ───*─────┼── ┤RX(-1.57079633)├ 
          ├──────────────┤ └────┘ ├───────────────┤ └────┘ ├───────────────┤ ├──────────────┤ ┌──┴─┐   │   ├───────────────┤ ┌──┴─┐   │   ├───────────────┤ 
q_2:  |0>─┤RX(1.57079633)├ ───*── ┤RX(-1.57079633)├ ───*── ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ┤CNOT├───┼── ┤RZ(-1.57079633)├ ┤CNOT├───┼── ┤RX(-1.57079633)├ 
          ├──────────────┤ ┌──┴─┐ ├───────────────┤ ┌──┴─

In [5]:
from pyqpanda3.core import RX, CPUQVM
for q in qubits:
    prog << RX(q, np.random.random())

machine = CPUQVM()
machine.run(prog, shots=1)
original_result = machine.result().get_prob_dict()

prog << circuit
machine.run(prog, shots=1)
final_result = machine.result().get_prob_dict()

for weight in range(5):
    prob_of_equal_weight_in_origin_result = sum(value for key, value in original_result.items() if key.count('1') == weight)
    prob_of_equal_weight_in_final_result =  sum(value for key, value in final_result.items() if key.count('1') == weight)

    print(f"Hamming weight {weight}: {prob_of_equal_weight_in_origin_result} -> {prob_of_equal_weight_in_final_result}")

Hamming weight 0: 0.823906046639297 -> 0.8239060466392997
Hamming weight 1: 0.1638360536206258 -> 0.1638360536206263
Hamming weight 2: 0.011885466874034448 -> 0.011885466874034509
Hamming weight 3: 0.00036840536662501565 -> 0.00036840536662501716
Hamming weight 4: 4.02749941773428e-06 -> 4.0274994177342665e-06
